Сделайте краткий обзор любой статьи посвященной тому или иному алгоритму для object detection, который не рассматривался на уроке. Проведите анализ: Чем отличается выбранная вами на рассмотрение архитектура нейронной сети от других архитектур? В чем плюсы и минусы данной архитектуры? Какие могут возникнуть трудности при применении данной архитектуры на практике?

Запустите детектор (ssdMobile_v2 или faster_rcnn, или любой другой детектор) для своей картинки и попробуйте найти 10 объектов, 100 объектов.

Исходная статья: https://arxiv.org/pdf/2105.04206.pdf
Алгоритм: YOLOR
Сурс: https://github.com/WongKinYiu/yolor

Кратко: вариант модели YOLO, в котором используется принцип заранее выведенного скрытого знания ("implicit knowledge") о той или иной категории объектов для уменьшения времени и количества примеров, нужного для обучения произвольной сети на распознавание данной категории объектов.

Плюс: если у нас есть модель implicit knowledge для нужной категории - можно не тратить много времени и не подыскивать много примеров для тренировки сети.

Минус: да где это implicit knowledge взять-то? Если для нужной категории его заранее нет - его придётся выводить. Кроме того, implicit knowledge - ещё одно потенциальное слабое место для атаки "отравлением данных", которая может задеть уже не одну модель, а все модели, пользующиеся данным implicit knowledge.

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import tensorflow_hub as hub

import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

import time
%matplotlib inline

In [3]:
def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img

def run_detector(detector, path, max_boxes=10):
  img = load_img(path)

  converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
  start_time = time.time()
  result = detector(converted_img)
  end_time = time.time()
  #print(result['detection_scores'])
  result = {key:value.numpy() for key,value in result.items()} 

  image_with_boxes = draw_boxes(
      img.numpy(), result["detection_boxes"],
      result["detection_class_entities"], result["detection_scores"], max_boxes=max_boxes)

  display_image(image_with_boxes)
  return result

In [4]:
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1" #@param ["https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1", "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"]

detector = hub.load(module_handle).signatures['default']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [5]:
import os, fnmatch
root_path = './'
listOfFiles = os.listdir(root_path)
pattern = '*.jpg'
List_name=[]
for entry in listOfFiles:
    if fnmatch.fnmatch(entry, pattern):
            List_name.append(np.array(root_path+entry))
List_name[0]

array('./725102.jpg', dtype='<U12')

In [ ]:
result = run_detector(detector, List_name[0], max_boxes=100)

При попытке найти 100 объектов на большом изображении ядро закономерно умирает -_-